In [13]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import FreqDist
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [2]:
nlp = pd.read_csv('nlp.csv')

In [3]:
nlp

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,...,main_cat,also_buy,also_view,similar_item,price,date,details,tech1,tech2,fit
0,0,2.0,False,"11 24, 2007",A1TOKUIUSJ77DU,B000PH9NNS,Gideon I. Gartner,"For former users of touch screens, this machin...",Tough Going,1195862400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
1,1,4.0,False,"11 21, 2007",A2SK1UQ9HI9ZYS,B000PH9NNS,Ernesto D. Jimenez Perez,The product is fantastic but the software that...,Excellent phone Software in Beta phase,1195603200,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
2,2,5.0,True,"11 19, 2007",A1DQVXBQATAK2R,B000PH9NNS,Aldo Bender,I bought this phone to replace my Nokia 9300 a...,A real smartphone,1195430400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
3,3,5.0,False,"09 25, 2007",A1PTDL2WFBL6GV,B000PH9NNS,ZANE Farooq,"I've used Nokia phones all my life, except wel...",Nokia...Wow,1190678400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
4,4,4.0,False,"08 21, 2007",A3QP1VWPQS9S9D,B000PH9NNS,Paolo M. Reyes,The Nokia E90 Communicator is a mobile office ...,"E90, Eager to transform the way people experie...",1187654400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10058,10058,1.0,True,"08 22, 2016",A3IESEK173F6ZZ,B01E6O625W,Kathleen Mulkey,"They are horrible, don't receive or make calls...",S4 sucks,1471824000,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN
10059,10059,5.0,True,"08 12, 2016",A1JCA9NI8WSAEL,B01E6O625W,dasun kloss,Awesome phone!! Works great! Delivered early!!...,Five Stars,1470960000,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN
10060,10060,5.0,True,"08 4, 2016",A3MYU5E9XUQJSO,B01E6O625W,Gail Roe,the phone work great got it before it was supp...,Five Stars,1470268800,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN
10061,10061,5.0,True,"07 19, 2016",ANWZUOXV9MDGS,B01E6O625W,Kenneth Darnell,Great phone and a good deal,Five Stars,1468886400,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN


In [10]:
## create target label
nlp['target'] = np.where(nlp.overall >3, 1,
                       np.where(nlp.overall <3.0, 0,
                       'unkown'))

In [11]:
nlp.drop(nlp[nlp['target'] =='unkown'].index, axis=0, inplace=True)
nlp.reset_index(drop=True, inplace=True)

nlp.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,...,also_view,similar_item,price,date,details,tech1,tech2,fit,clean_up_review,target
0,0,2.0,False,"11 24, 2007",A1TOKUIUSJ77DU,B000PH9NNS,Gideon I. Gartner,"for former users of touch screens, this machin...",Tough Going,1195862400,...,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,for former users of touch screens this machine...,0
1,1,4.0,False,"11 21, 2007",A2SK1UQ9HI9ZYS,B000PH9NNS,Ernesto D. Jimenez Perez,the product is fantastic but the software that...,Excellent phone Software in Beta phase,1195603200,...,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,the product is fantastic but the software that...,1
2,2,5.0,True,"11 19, 2007",A1DQVXBQATAK2R,B000PH9NNS,Aldo Bender,i bought this phone to replace my nokia 9300 a...,A real smartphone,1195430400,...,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,i bought this phone to replace my nokia 9300 a...,1
3,3,5.0,False,"09 25, 2007",A1PTDL2WFBL6GV,B000PH9NNS,ZANE Farooq,"i've used nokia phones all my life, except wel...",Nokia...Wow,1190678400,...,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,i've used nokia phones all my life except well...,1
4,4,4.0,False,"08 21, 2007",A3QP1VWPQS9S9D,B000PH9NNS,Paolo M. Reyes,the nokia e90 communicator is a mobile office ...,"E90, Eager to transform the way people experie...",1187654400,...,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,the nokia e90 communicator is a mobile office ...,1


In [14]:
nlp.reviewText.dropna(inplace = True)

In [15]:
## clean up review column
nlp.reviewText = nlp.reviewText.str.lower()
nlp['clean_up_review']= nlp.reviewText.str.replace('\n',''). \
                                          str.replace('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')
nlp.reset_index(drop=True, inplace=True)

In [16]:
## split dataset
nlp_train, nlp_test = train_test_split(nlp[['clean_up_review','target']],test_size=0.3)

In [17]:
nlp_train.clean_up_review

6105    this phone is lock but at the time of my order...
861                                                  good
5729                                              todo ok
6592    i hope samsung know that how bad and inconveni...
1528    i purchased this lg smartphone after many hour...
                              ...                        
3374    this windows phone disapoints me i thought i w...
4826    i've only had a little time to use this so thi...
8590    i accidentally drowned my lg g5 and needed a r...
5990    the phone was fine except it didn't have a sim...
8789    exactly as promised a new from apple iphone se...
Name: clean_up_review, Length: 6595, dtype: object

In [18]:
import eli5

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

my_stop_words = text.ENGLISH_STOP_WORDS.union(["good","great","best","better","love","junk","suck","worst","bad","love"])

C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [19]:
#count vectorizer
lr_model = LogisticRegression(C=5)
vec = CountVectorizer(stop_words = 'english')
pipe = make_pipeline(vec, lr_model)
pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
train_f1, test_f1  

C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(0.9921152388172858, 0.9020162716660771)

In [20]:
#tfidf vectorizer
lr_model = LogisticRegression(C=5)
vec = TfidfVectorizer(stop_words = 'english', min_df = 5,lowercase = True, ngram_range = (1,3))
pipe = make_pipeline(vec, lr_model)
pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
train_f1, test_f1 

(0.9763457164518574, 0.9112133003183587)

In [25]:
## lr model with TfidfVectorizer
C = np.arange(0.5,10,0.5)
max_test_f1 = 0
for i in C:
    for j in range(2, 10, 1):
        lr_model = LogisticRegression(C=i)
        #IF need to update the stop word, use the follow code.
        #my_stop_words = text.ENGLISH_STOP_WORDS.union(["love",'great','best','like','better','phone'])
        vec = TfidfVectorizer(min_df = j, stop_words ='english', lowercase = True, ngram_range = (1,3))
        pipe = make_pipeline(vec, lr_model)
        pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
        train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
        train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
        test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
        test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
        train_f1, test_f1
        print('train_f1:',train_f1,'test_f1:', test_f1, ",","C", i, "min_df",  j)
        if( test_f1 > max_test_f1):
            opt_n = i
            opt_min_df = j 
            max_test_f1 = test_f1

train_f1: 0.9120545868081881 test_f1: 0.8832684824902723 , C 0.5 min_df 2
train_f1: 0.9155420773313117 test_f1: 0.8878669968164131 , C 0.5 min_df 3
train_f1: 0.9158453373768006 test_f1: 0.8903431199151044 , C 0.5 min_df 4
train_f1: 0.9161485974222896 test_f1: 0.8899893880438627 , C 0.5 min_df 5
train_f1: 0.9163002274450341 test_f1: 0.8896356561726212 , C 0.5 min_df 6
train_f1: 0.9163002274450341 test_f1: 0.8896356561726212 , C 0.5 min_df 7
train_f1: 0.9169067475360121 test_f1: 0.8903431199151044 , C 0.5 min_df 8
train_f1: 0.9170583775587566 test_f1: 0.8917580474000707 , C 0.5 min_df 9
train_f1: 0.9429871114480667 test_f1: 0.9002476123098692 , C 1.0 min_df 2
train_f1: 0.9417740712661107 test_f1: 0.9023700035373187 , C 1.0 min_df 3
train_f1: 0.9388931008339652 test_f1: 0.9048461266360099 , C 1.0 min_df 4
train_f1: 0.9381349507202426 test_f1: 0.9059073222497347 , C 1.0 min_df 5
train_f1: 0.9379833206974981 test_f1: 0.9083834453484259 , C 1.0 min_df 6
train_f1: 0.9370735405610311 test_f1: 

C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.9866565579984837 test_f1: 0.9133356915458083 , C 6.0 min_df 3
train_f1: 0.9827141774071266 test_f1: 0.9122744959320835 , C 6.0 min_df 4
train_f1: 0.980439727065959 test_f1: 0.9115670321896003 , C 6.0 min_df 5
train_f1: 0.97710386656558 test_f1: 0.9101521047046339 , C 6.0 min_df 6
train_f1: 0.974981046247157 test_f1: 0.9087371772196675 , C 6.0 min_df 7
train_f1: 0.9734647460197119 test_f1: 0.9094446409621507 , C 6.0 min_df 8
train_f1: 0.9724033358605004 test_f1: 0.9080297134771843 , C 6.0 min_df 9
train_f1: 0.9916603487490523 test_f1: 0.9136894234170498 , C 6.5 min_df 2


C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.9877179681576952 test_f1: 0.9122744959320835 , C 6.5 min_df 3
train_f1: 0.9845337376800607 test_f1: 0.9115670321896003 , C 6.5 min_df 4
train_f1: 0.9819560272934041 test_f1: 0.9115670321896003 , C 6.5 min_df 5
train_f1: 0.978013646702047 test_f1: 0.9108595684471171 , C 6.5 min_df 6
train_f1: 0.9763457164518574 test_f1: 0.9090909090909091 , C 6.5 min_df 7


...

train_f1: 0.9849886277482942 test_f1: 0.9112133003183587 , C 9.5 min_df 6


C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.9836239575435937 test_f1: 0.9087371772196675 , C 9.5 min_df 7
train_f1: 0.9811978771796815 test_f1: 0.9062610541209764 , C 9.5 min_df 8


C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.979833206974981 test_f1: 0.9076759816059426 , C 9.5 min_df 9


In [27]:
print(opt_n, opt_min_df, max_test_f1)

1.5 6 0.9136894234170498


In [28]:
## lr model with CountVectorizer
C = np.arange(0.5,10,0.5)
max_test_f1 = 0
for i in C:
    for j in range(2, 10, 1):
        lr_model = LogisticRegression(C=i)
        #my_stop_words = text.ENGLISH_STOP_WORDS.union(["love",'great','best','like','better','phone'])
        vec = CountVectorizer(min_df = j, stop_words ='english', lowercase = True, ngram_range = (1,3))
        pipe = make_pipeline(vec, lr_model)
        pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
        train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
        train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
        test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
        test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
        train_f1, test_f1
        print('train_f1:',train_f1,'test_f1:', test_f1, ",","C", i, "min_df",  j)
        if( test_f1 > max_test_f1):
            opt_c = i
            opt_min_df = j 
            max_test_f1 = test_f1

train_f1: 0.978013646702047 test_f1: 0.8967102935974531 , C 0.5 min_df 2
train_f1: 0.9742228961334344 test_f1: 0.8974177573399363 , C 0.5 min_df 3
train_f1: 0.9704321455648218 test_f1: 0.8977714892111779 , C 0.5 min_df 4
train_f1: 0.9684609552691433 test_f1: 0.8956490979837284 , C 0.5 min_df 5
train_f1: 0.9661865049279758 test_f1: 0.8956490979837284 , C 0.5 min_df 6
train_f1: 0.9664897649734647 test_f1: 0.894234170498762 , C 0.5 min_df 7
train_f1: 0.9643669446550417 test_f1: 0.8956490979837284 , C 0.5 min_df 8
train_f1: 0.9625473843821076 test_f1: 0.8952953661124867 , C 0.5 min_df 9
train_f1: 0.9878695981804397 test_f1: 0.9037849310222851 , C 1.0 min_df 2


C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.9851402577710386 test_f1: 0.9030774672798019 , C 1.0 min_df 3


C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.9824109173616377 test_f1: 0.9027237354085603 , C 1.0 min_df 4
train_f1: 0.9799848369977255 test_f1: 0.9013088079235939 , C 1.0 min_df 5


...

train_f1: 0.9931766489764974 test_f1: 0.8935267067562788 , C 9.5 min_df 7


C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.9927217589082639 test_f1: 0.8952953661124867 , C 9.5 min_df 8


C:\Users\a5694\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.9918119787717968 test_f1: 0.8906968517863459 , C 9.5 min_df 9


In [21]:
### rf model with CountVectorizer
rf_model = RandomForestClassifier(n_estimators=1000,min_samples_split=3)
vec = CountVectorizer(stop_words = 'english', min_df = 5,lowercase = True, ngram_range = (1,3))
pipe = make_pipeline(vec, rf_model)
pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
train_f1, test_f1 

(0.9956027293404094, 0.8945879023700035)

In [22]:
#rf tfidf
rf_model = RandomForestClassifier(n_estimators=100)
vec = TfidfVectorizer(stop_words = 'english', min_df = 5,lowercase = True, ngram_range = (1,3))
pipe = make_pipeline(vec, rf_model)
pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
train_f1, test_f1 

(0.9962092494313874, 0.8931729748850371)

In [23]:
## rf model with TfidfVectorizer
n_estimater = np.arange(100,1000,100)
max_test_f1 = 0
for i in n_estimater:
    for j in range(2, 10, 1):
        rf_model = RandomForestClassifier(n_estimators=i)
        #my_stop_words = text.ENGLISH_STOP_WORDS.union(["love",'great','best','like','better','phone'])
        vec = TfidfVectorizer(min_df = j, stop_words ='english', lowercase = True, ngram_range = (1,3))
        pipe = make_pipeline(vec, rf_model)
        pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
        train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
        train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
        test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
        test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
        train_f1, test_f1
        print('train_f1:',train_f1,'test_f1:', test_f1, ",","n_estimator", i, "min_df",  j)
        if( test_f1 > max_test_f1):
            opt_n = i
            opt_min_df = j 
            max_test_f1 = test_f1

train_f1: 0.9968157695223654 test_f1: 0.8931729748850371 , n_estimator 100 min_df 2
train_f1: 0.996664139499621 test_f1: 0.8921117792713124 , n_estimator 100 min_df 3
train_f1: 0.9963608794541319 test_f1: 0.8931729748850371 , n_estimator 100 min_df 4
train_f1: 0.9962092494313874 test_f1: 0.8945879023700035 , n_estimator 100 min_df 5
train_f1: 0.9962092494313874 test_f1: 0.8875132649451716 , n_estimator 100 min_df 6
train_f1: 0.9962092494313874 test_f1: 0.8910505836575876 , n_estimator 100 min_df 7
train_f1: 0.9962092494313874 test_f1: 0.8924655111425539 , n_estimator 100 min_df 8
train_f1: 0.9960576194086429 test_f1: 0.8892819243013795 , n_estimator 100 min_df 9
train_f1: 0.9969673995451099 test_f1: 0.8931729748850371 , n_estimator 200 min_df 2
train_f1: 0.996664139499621 test_f1: 0.8924655111425539 , n_estimator 200 min_df 3
train_f1: 0.9963608794541319 test_f1: 0.8928192430137956 , n_estimator 200 min_df 4
train_f1: 0.9962092494313874 test_f1: 0.8956490979837284 , n_estimator 200 min

In [194]:
print(opt_n, opt_min_df, max_test_f1)

500 4 0.9006013441811107


In [29]:
## rf model with CountVectorizer
nestimator = np.arange(100,1000,100)
max_test_f1 = 0
for i in nestimator:
    for j in range(2, 10, 1):
        rf_model = RandomForestClassifier(n_estimators=i)
        vec = CountVectorizer(min_df = j, stop_words ='english', lowercase = True, ngram_range = (1,3))
        pipe = make_pipeline(vec, rf_model)
        pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
        train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
        train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
        test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
        test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
        train_f1, test_f1
        print('train_f1:',train_f1,'test_f1:', test_f1, ",","C", i, "min_df",  j)
        if( test_f1 > max_test_f1):
            opt_rf_cnt_n = i
            opt_rf_cnt_min_df = j 
            max_test_rf_cnt_f1 = test_f1

train_f1: 0.9969673995451099 test_f1: 0.8928192430137956 , C 100 min_df 2
train_f1: 0.996664139499621 test_f1: 0.8952953661124867 , C 100 min_df 3
train_f1: 0.9963608794541319 test_f1: 0.8949416342412452 , C 100 min_df 4
train_f1: 0.9962092494313874 test_f1: 0.8970640254686947 , C 100 min_df 5
train_f1: 0.9962092494313874 test_f1: 0.8938804386275203 , C 100 min_df 6
train_f1: 0.9962092494313874 test_f1: 0.8892819243013795 , C 100 min_df 7
train_f1: 0.9962092494313874 test_f1: 0.894234170498762 , C 100 min_df 8
train_f1: 0.9960576194086429 test_f1: 0.8935267067562788 , C 100 min_df 9
train_f1: 0.9969673995451099 test_f1: 0.8928192430137956 , C 200 min_df 2
train_f1: 0.996664139499621 test_f1: 0.8967102935974531 , C 200 min_df 3
train_f1: 0.9963608794541319 test_f1: 0.8974177573399363 , C 200 min_df 4
train_f1: 0.9962092494313874 test_f1: 0.8952953661124867 , C 200 min_df 5
train_f1: 0.9962092494313874 test_f1: 0.894234170498762 , C 200 min_df 6
train_f1: 0.9962092494313874 test_f1: 0.89

In [30]:
print(opt_rf_cnt_n,opt_rf_cnt_min_df,max_test_lr_cnt_f1)

900 9 0.8910505836575876
